## Data 
* Data from https://opendata.dwd.de/climate_environment/health/historical_alerts/heat_warnings/
* Geodata from https://maps.dwd.de dwd:Warngebiete_Kreise as WMS GeoJSON VectorTileswith data modified from © GeoBasis-DE / BKG, Statistisches Bundesamt (Destatis) (2021)

In [42]:
using DataFrames
using CSV
using GLMakie,GeoMakie

In [43]:
import GeoDataFrames

In [44]:
boundary=GeoDataFrames.read("heat_warnings/dwd-Warngebiete_Kreise.geojson")

Row,geometry,id,BL,CCC,COPYRIGHT,FID,KURZNAME,MAX_HEIGHT,MIN_HEIGHT,NAME,RZ,STATE,TYPE,WARNCELLID,WARNCENTER
,IGeometry,String,String,String,String,Int32,Union…,Int32,Int32,String,String,Int32,Int32,Int32,Int32
1,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2506,NI,PEX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),17,Peine,132,52,Kreis Peine,HA,3,1,103157000,201
2,Geometry: wkbMultiPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2505,NI,WFX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),18,Wolfenbüttel,322,73,Kreis Wolfenbüttel,HA,3,1,103158000,201
3,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2504,NI,GOE,© GeoBasis-DE / BKG 2021 (Daten modifiziert),19,Göttingen,863,108,Kreis Göttingen,HA,3,1,103159000,201
4,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2503,NI,HAN,© GeoBasis-DE / BKG 2021 (Daten modifiziert),20,Hannover,402,18,Region Hannover,HA,3,1,103241000,201
5,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2502,NI,DHX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),21,Diepholz,159,2,Kreis Diepholz,HA,3,1,103251000,201
6,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2501,NI,HMX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),22,Hameln-Pyrmont,438,55,Kreis Hameln-Pyrmont,HA,3,1,103252000,201
7,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-2500,NI,HIX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),23,Hildesheim,479,57,Kreis Hildesheim,HA,3,1,103254000,201
8,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-24ff,NI,HOL,© GeoBasis-DE / BKG 2021 (Daten modifiziert),24,Holzminden,524,69,Kreis Holzminden,HA,3,1,103255000,201
9,Geometry: wkbPolygon,Warngebiete_Kreise.fid--2828b967_18b444abff3_-24fe,SH,FLX,© GeoBasis-DE / BKG 2021 (Daten modifiziert),1,Flensburg,63,0,Stadt Flensburg,HA,1,1,101001000,201


In [262]:
for year ∈ 2005:2023
#year=2023
    df = CSV.File("heat_warnings/heat_alerts_$year.csv") |> DataFrame;
    heat_alerts=combine(groupby(df,[:Name,:Warnstufe]),nrow)
    colors=[]
    for bound ∈ eachrow(boundary)
        warnings=heat_alerts[heat_alerts.Name.==bound.NAME,:]
        if size(warnings,1)==0
            push!(colors, cols[1][1])
            continue
        end
        warning_1=warnings[warnings.Warnstufe.==1,:]
        xe=0
        if size(warning_1,1)==0
            xe=1
        else
            xe=warning_1.nrow[1]
        end
        warning_3=warnings[warnings.Warnstufe.==3,:]
        ye=0
        if size(warning_3,1)==0
            ye=1
        else
            ye=warning_3.nrow[1]
        end
        xe=findfirst(xe.<bounds_x)
        ye=findfirst(ye.<bounds_y)
        push!(colors,cols[ye][xe])
        
    end
    f=Figure(resolution=(1400,800))
    ax=GeoAxis(f[1:2,1:3], latlims=(46.9,55.1),lonlims=(5.5,15.5),title="Number of days in year $year with heat alerts")
    p=poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(boundary.geometry)),color=colors, strokewidth=0.1, strokecolor=:black)
#Colorbar(f[1,2],p,label="Heat alerts per county in year $(year)")

hidedecorations!(ax)
hidespines!(ax)
gf=f[2,4]#GridLayout(2,2)
ax=Axis(gf,xlabel="Warning of high heat exposure", ylabel="Warning of extreme heat exposure",xticks=(1:5,["0";string.(bounds_x)[1:4]]),yticks=(1:5,["0";string.(bounds_y)[1:4]]),aspect=1)
for x in 1:5
    for y in 1:5
    poly!(ax,Rect(x,y, 1, 1), color=cols[y][x])
    end
end

hidedecorations!(ax,label = false, ticklabels = false, ticks = false)
hidespines!(ax)
#Label(f[4,1:3],text="Data: © Deutscher Wetterdienst\nGeodata: © Data modified from © GeoBasis-DE / BKG, Statistisches Bundesamt (Destatis) (2021))",justification=:left)

Makie.resize_to_layout!(f)
save("heat_alerts_sum_ldk_$(year).png",f,px_per_unit=2)
end

In [183]:
f

In [187]:
bounds_x=[5,10,15,20,100]
bounds_y=[1,2,3,5,100]

5-element Vector{Int64}:
   1
   2
   3
   5
 100

In [129]:
using Colors,ColorSchemes

In [130]:
using CairoMakie

In [223]:
cols=[cgrad(:managua10,5,categorical=true,rev=true),cgrad(:bamako10, 5,categorical=true),cgrad(:turku10,5,categorical=true),cgrad(ColorSchemes.acton10, 5,categorical=true),cgrad(:buda10,5,categorical=true)]

5-element Vector{PlotUtils.CategoricalColorGradient}:
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.5039,0.9023,0.9961,1.0), RGBA{Float64}(0.3555,0.5,0.7344,1.0), RGBA{Float64}(0.3047,0.1914,0.3672,1.0), RGBA{Float64}(0.5703,0.2734,0.2305,1.0), RGBA{Float64}(0.8633,0.6016,0.332,1.0)], "", ""), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.0547,0.2617,0.2461,1.0), RGBA{Float64}(0.2109,0.3672,0.1484,1.0), RGBA{Float64}(0.4492,0.5117,0.0078,1.0), RGBA{Float64}(0.7422,0.6562,0.1797,1.0), RGBA{Float64}(0.9961,0.8945,0.6758,1.0)], "", ""), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.1406,0.1406,0.125,1.0), RGBA{Float64}(0.3711,0.3711,0.2656,1.0), RGBA{Float64}(0.6602,0.5977,0.3945,1.0), RGBA{Float64}(0.9141,0.6758,0.5

In [232]:
cols=[cgrad(:grayC,5,categorical=true),cgrad(:bamako10,5,categorical=true),cgrad(:berlin10, 5,categorical=true),cgrad(:lipari, 10,categorical=true)[5:10],cgrad(:buda10,5,categorical=true)]


5-element Vector{Any}:
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.0,0.0,0.0,1.0), RGBA{Float64}(0.26385000000000003,0.26385000000000003,0.26385000000000003,1.0), RGBA{Float64}(0.46635,0.46635,0.46635,1.0), RGBA{Float64}(0.6862250000000001,0.6862250000000001,0.6862250000000001,1.0), RGBA{Float64}(1.0,1.0,1.0,1.0)], "", ""), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.0547,0.2617,0.2461,1.0), RGBA{Float64}(0.2109,0.3672,0.1484,1.0), RGBA{Float64}(0.4492,0.5117,0.0078,1.0), RGBA{Float64}(0.7422,0.6562,0.1797,1.0), RGBA{Float64}(0.9961,0.8945,0.6758,1.0)], "", ""), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.3555,0.6406,0.8555,1.0), RGBA{Float64}(0.1016,0.2578,0.3359,1.0), RGBA{Float64}(0.1562,0.0508,0.0039,1.0

In [242]:
cols=[cgrad(:grayC,5,categorical=true),cgrad(:lajolla,10,categorical=true)[1:5],cgrad(:lapaz,10,categorical=true)[1:5],cgrad(:lajolla,10,categorical=true)[6:10],cgrad(:lapaz,10,categorical=true)[6:10]]

5-element Vector{Any}:
 PlotUtils.CategoricalColorGradient(ColorScheme{Vector{RGBA{Float64}}, String, String}(RGBA{Float64}[RGBA{Float64}(0.0,0.0,0.0,1.0), RGBA{Float64}(0.26385000000000003,0.26385000000000003,0.26385000000000003,1.0), RGBA{Float64}(0.46635,0.46635,0.46635,1.0), RGBA{Float64}(0.6862250000000001,0.6862250000000001,0.6862250000000001,1.0), RGBA{Float64}(1.0,1.0,1.0,1.0)], "", ""), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
 RGBA{Float64}[RGBA{Float64}(0.0988,0.0997,0.0001,1.0), RGBA{Float64}(0.20123333333333332,0.13546666666666668,0.0601,1.0), RGBA{Float64}(0.35353333333333337,0.18706666666666666,0.13706666666666664,1.0), RGBA{Float64}(0.5626,0.2525,0.2385,1.0), RGBA{Float64}(0.7827,0.31379999999999997,0.2926333333333333,1.0)]
 RGBA{Float64}[RGBA{Float64}(0.1035,0.0478,0.3935,1.0), RGBA{Float64}(0.13623333333333332,0.17969999999999997,0.4828,1.0), RGBA{Float64}(0.16519999999999999,0.2952666666666667,0.5597333333333333,1.0), RGBA{Float64}(0.2117,0.4044,0.6147,1.0), RGBA{Float64}(0.2

In [198]:
cgrad(:lisbon10,5,categorical=true)

In [205]:
cgrad(:managua10,5,categorical=true,rev=true)


In [202]:
cgrad(:lajolla10, 5,categorical=true)

In [201]:
cgrad(:lipari10, 5,categorical=true)



In [214]:
cgrad(:berlin, 25,categorical=true)
